# 퇴근시간 버스 승차 인원 예측

In [21]:
train = read.csv("C:/Users/MEDICI/Downloads/bus_data/train.csv",fileEncoding='utf-8',stringsAsFactors = F)
test = read.csv("C:/Users/MEDICI/Downloads/bus_data/test.csv",fileEncoding='utf-8',stringsAsFactors = F)
bts = read.csv("C:/Users/MEDICI/Downloads/bus_data/bus_bts.csv",fileEncoding='utf-8')

In [22]:
head(train)

id,date,bus_route_id,in_out,station_code,station_name,latitude,longitude,X6.7_ride,X7.8_ride,...,X9.10_ride,X10.11_ride,X11.12_ride,X6.7_takeoff,X7.8_takeoff,X8.9_takeoff,X9.10_takeoff,X10.11_takeoff,X11.12_takeoff,X18.20_ride
0,2019-09-01,4270000,시외,344,제주썬호텔,33.48990,126.4937,0,1,...,5,2,6,0,0,0,0,0,0,0
1,2019-09-01,4270000,시외,357,한라병원,33.48944,126.4851,1,4,...,2,5,6,0,0,0,0,0,0,5
2,2019-09-01,4270000,시외,432,정존마을,33.48181,126.4735,1,1,...,2,0,0,0,0,0,0,0,0,2
3,2019-09-01,4270000,시내,1579,제주국제공항(600번),33.50577,126.4925,0,17,...,26,14,16,0,0,0,0,0,0,53
4,2019-09-01,4270000,시내,1646,중문관광단지입구,33.25579,126.4126,0,0,...,0,0,0,0,0,0,1,0,0,0
5,2019-09-01,4270000,시내,1648,예래입구,33.25862,126.4044,0,0,...,0,1,0,0,0,1,0,1,0,0


In [41]:
head(test)

id,date,bus_route_id,in_out,station_code,station_name,latitude,longitude,X6.7_ride,X7.8_ride,...,r1012,t68,t810,t1012,dis_jeju,dis_gosan,dis_seongsan,dis_po,dist_min,dis_name
415423,2019-10-01,4270000,1,344,제주썬호텔,33.48990,126.4937,4,4,...,10,0,0,1,4286.659,37692.77,37713.63,27841.88,4286.659,0
415424,2019-10-01,4270000,1,357,한라병원,33.48944,126.4851,1,6,...,19,0,0,0,4966.848,37008.49,38465.82,27996.18,4966.848,0
415425,2019-10-01,4270000,1,432,정존마을,33.48181,126.4735,2,4,...,3,0,0,0,6330.746,35642.13,39259.82,27496.55,6330.746,0
415426,2019-10-01,4270000,0,1579,제주국제공항(600번),33.50577,126.4925,1,11,...,46,0,0,0,3575.272,38643.40,38386.80,29579.40,3575.272,0
415427,2019-10-01,4270000,0,1636,롯데호텔,33.24872,126.4103,0,0,...,0,0,1,0,31460.686,23594.01,46355.30,14445.61,14445.605,0
415428,2019-10-01,4270000,0,1646,중문관광단지입구,33.25579,126.4126,0,0,...,0,0,2,0,30652.098,23648.36,45899.14,14269.79,14269.792,0


In [3]:
head(bts)

user_card_id,bus_route_id,vhc_id,geton_date,geton_time,geton_station_code,geton_station_name,getoff_date,getoff_time,getoff_station_code,getoff_station_name,user_category,user_count
1.01001e+15,23000000,149793674,2019-09-10,06:34:45,360,노형오거리,2019-09-10,07:10:31,592,화북초등학교,1,1
1.01001e+15,23000000,149793674,2019-09-10,06:34:58,360,노형오거리,2019-09-10,06:56:27,3273,고산동산(광양방면),1,1
1.01916e+15,21420000,149793535,2019-09-10,07:19:07,2495,동광환승정류장4(제주방면),2019-09-10,07:40:29,431,정존마을,4,1
1.01915e+15,21420000,149793512,2019-09-09,09:14:47,3282,대정환승정류장(대정읍사무소),2019-09-09,10:02:46,431,정존마을,1,1
1.01001e+15,21420000,149793512,2019-09-09,09:28:53,2820,삼정지에듀,2019-09-09,10:21:37,2972,제주국제공항(종점),4,1
1.01915e+15,31520000,149793453,2019-09-10,07:54:08,189,도호동,2019-09-10,08:07:09,3539,탐라도서관,4,1


## 데이터 전처리

In [2]:
library('lubridate')

Warning message:
"package 'lubridate' was built under R version 3.6.3"
Attaching package: 'lubridate'

The following objects are masked from 'package:base':

    date, intersect, setdiff, union



In [3]:
train$date = as.Date(train$date)
train['weekday'] = wday(train$date)

test$date = as.Date(test$date)
test['weekday'] = wday(test$date)

dim(train)
dim(test)

[1] 415423     22

[1] 228170     21

In [27]:
train[train$in_out == "시외",'in_out'] <- 1
train[train$in_out == "시내",'in_out'] <- 0

In [25]:
test[test$in_out == "시외",'in_out'] <- 1
test[test$in_out == "시내",'in_out'] <-  0

In [29]:
train['r68']=train['X6.7_ride']+train['X7.8_ride'] # 6 ~ 8시 승차인원
train['r810']=train['X8.9_ride']+train['X9.10_ride']
train['r1012']=train['X10.11_ride']+train['X11.12_ride']

train['t68']=train['X6.7_takeoff']+train['X7.8_takeoff'] # 6 ~ 8시 하차인원
train['t810']=train['X8.9_takeoff']+train['X9.10_takeoff']
train['t1012']=train['X10.11_takeoff']+train['X11.12_takeoff']

In [30]:
test['r68']=test['X6.7_ride']+test['X7.8_ride'] # 6 ~ 8시 승차인원
test['r810']=test['X8.9_ride']+test['X9.10_ride']
test['r1012']=test['X10.11_ride']+test['X11.12_ride']

test['t68']=test['X6.7_takeoff']+test['X7.8_takeoff'] # 6 ~ 8시 하차인원
test['t810']=test['X8.9_takeoff']+test['X9.10_takeoff']
test['t1012']=test['X10.11_takeoff']+test['X11.12_takeoff']

### 원핫인코딩

In [ ]:
library('caret')
dmy <- dummyVars(~., data = train)
n_train <- data.frame(predict(dmy, newdata = train))

dmy <- dummyVars(~., data = test)
n_test <- data.frame(predict(dmy, newdata = test))

### 지역별 거리

In [33]:
install.packages("geosphere")


  There is a binary version available but the source version is later:
          binary source needs_compilation
geosphere 1.5-10 1.5-14              TRUE

  Binaries will be installed
package 'geosphere' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\MEDICI\AppData\Local\Temp\RtmpsFGeV2\downloaded_packages


In [34]:
library("geosphere")
jeju=c( 126.52969,33.51411) # 제주 측정소 근처
gosan=c(126.16283 ,33.29382 ) #고산 측정소 근처
seongsan=c(126.8802,33.38677 ) #성산 측정소 근처
po=c(126.5653,33.24616 ) #서귀포 측정소 근처

Warning message:
"package 'geosphere' was built under R version 3.6.3"

In [35]:
train['dis_jeju']=0
train['dis_gosan']=0
train['dis_seongsan']=0
train['dis_po']=0
for(i in 1:nrow(train)){
  train$dis_jeju[i]= distGeo(jeju, c(train$longitude[i], train$latitude[i]))
  train$dis_gosan[i]= distGeo(gosan, c(train$longitude[i], train$latitude[i]))
  train$dis_seongsan[i]= distGeo(seongsan, c(train$longitude[i], train$latitude[i]))
  train$dis_po[i]= distGeo(po, c(train$longitude[i], train$latitude[i]))

}

In [36]:
a = cbind(train['dis_jeju'] ,train['dis_gosan'],train['dis_seongsan'] ,train['dis_po'])
train['dist_min'] = apply(a,1, min)
train['dis_name']=0

In [37]:
train['dis_jeju'] = 0
train['dis_jeju'] = 0
train['dis_jeju'] = 0
train['dis_jeju'] = 0

In [42]:
for(i in 1:nrow(train)){
  if(train$dis_jeju[i] == train$dist_min[i]){
    train$dis_name[i] = 1
  }else if(train$dis_gosan[i] == train$dist_min[i]){
    train$dis_name[i] = 2
  }else if(train$dis_po[i] == train$dist_min[i]){
    train$dis_name[i] = 3
  }else if(train$dis_seongsan[i] == train$dist_min[i]){
    train$dis_name[i] = 4
  }
}

In [43]:
for(i in 1:nrow(train)){
  if(train$dis_name [i] == '1'){
    train$dis_jeju[i] <- 1
  }else if(train$dis_name [i] == '2'){
    train$dis_gosan[i] <- 1
  }else if(train$dis_name [i] == '3'){
    train$dis_po[i] <- 1
  }else if(train$dis_name [i] == '4'){
    train$dis_seongsan[i] <- 1
  }
}

In [39]:
test['dis_jeju']=0
test['dis_gosan']=0
test['dis_seongsan']=0
test['dis_po']=0
for(i in 1:nrow(test)){
  test$dis_jeju[i]= distGeo(jeju, c(test$longitude[i], test$latitude[i]))
  test$dis_gosan[i]= distGeo(gosan, c(test$longitude[i], test$latitude[i]))
  test$dis_seongsan[i]= distGeo(seongsan, c(test$longitude[i], test$latitude[i]))
  test$dis_po[i]= distGeo(po, c(test$longitude[i], test$latitude[i]))
  
}

In [40]:
b = cbind(test['dis_jeju'] ,test['dis_gosan'],test['dis_seongsan'] ,test['dis_po'])
test['dist_min'] = apply(b,1, min)
test['dis_name']=0

In [ ]:
for(i in 1:nrow(test)){
  if(test$dis_jeju[i] == test$dist_min[i]){
    test$dis_name[i] = 1
  }else if(test$dis_gosan[i] == test$dist_min[i]){
    test$dis_name[i] = 2
  }else if(test$dis_po[i] == test$dist_min[i]){
    test$dis_name[i] = 3
  }else if(test$dis_seongsan[i] == test$dist_min[i]){
    test$dis_name[i] = 4
  }
}

In [ ]:
test['dis_jeju']=0
test['dis_gosan']=0
test['dis_seongsan']=0
test['dis_po']=0
for(i in 1:nrow(test)){
  if(test$dis_name [i] == '1'){
    test$dis_jeju[i] <- 1
  }else if(test$dis_name [i] == '2'){
    test$dis_gosan[i] <- 1
  }else if(test$dis_name [i] == '3'){
    test$dis_po[i] <- 1
  }else if(test$dis_name [i] == '4'){
    test$dis_seongsan[i] <- 1
  }
}

## 변수 관계 확인

In [ ]:
except <- c(nearZeroVar(train))
tre <- train[-c(except,1)]
tre <- tre[,-c(1,4)]
correlations <- cor(tre)
# 상관관계가 높은 것은 데이터를 걸러낸다. 
highCorr <- findCorrelation(correlations, cutoff = .75) 
tre <- tre[, -highCorr]    

# 아래에 상관계수가 높은 데이터 셋을 걸러내어 데이터가 나왔다.  
glimpse(tre)

fitControl <- trainControl(method = "repeatedcv", number = 10, repeats = 5)
rf_fit <- train(X18.20_ride ~ ., data = tre, method = "rf", trControl = fitControl, verbose = F)
rf_fit

## 머신러닝

### lm

In [ ]:
input_var = c('in_out' ,'station_code','latitude','longitude','weekday','r810', 'r1012', 'r68', 't68', 't810','t1012','dis_name')
target = c('X18.20_ride')

In [ ]:
X_train <- train[input_var]
y_train <- train[target]
X_test <- test[input_var]

In [ ]:
tr <- train[c('X18.20_ride','in_out' ,'station_code','r810', 'r1012', 'r68', 't68', 't810','t1012','Mon','Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun', 'dis_jeju', 'dis_gosan', 'dis_seongsan', 'dis_po')]
test <- test[c('X18.20_ride','in_out' ,'station_code','r810', 'r1012', 'r68', 't68', 't810','t1012','Mon','Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun', 'dis_jeju', 'dis_gosan', 'dis_seongsan', 'dis_po')]

model_lm <- lm(X18.20_ride ~  ., data = tr)
summary(model_lm)

In [ ]:
install.packages('forecast')
library(forecast)
accuracy(model_lm)

### 다중공선성 확인

In [ ]:
install.packages("car")
library(car)
vif(model_lm)

### RandomForest

In [ ]:
#install.packages('randomForest')
library(randomForest)
forest_m <- randomForest(X18.20_ride ~ ., data=tr)
forest_m$predicted 
forest_m$importance
y_pred = predict(forest_m, te)

In [ ]:
memory.size(max = TRUE)    # OS에서 얻은 최대 메모리 크기 = OS로부터 R이 사용 가능한 메모리
memory.size(max = FALSE)   # 현재 사용중인 메모리 크기
memory.limit(size = NA)    # 컴퓨터의 최대 메모리 한계치 

# 컴퓨터의 최대 메모리 한계치 약 49GB로 높이기
memory.limit(size = 50000)   

### SKlearn.linearRegression

In [ ]:
#install.packages('reticulate')
library(reticulate)
py_config()
use_python('C:/Users/MEDICI/Anaconda3/envs/r-reticulate/python.exe')
sk <- import('sklearn.linear_model')
sk.met = import('sklearn.metrics')

In [ ]:
# 모델 형성
myModel = sk.met$LinearRegression()
# 모델 학습
myModel$fit(X_train, y_train)
# 모델 예측
y_pred = myModel$predict(X_train)
sk.met$r2_score( tr$X18.20_ride , y_pred)

In [ ]:
#모델평가
lr_r = sk.met$r2_score(y_train, lr_pred)
lr_r

lr_rmse = sk.met$mean_squared_error(y,lr_pred)^0.5
lr_rmse

In [ ]:
lr_model$summary

### sklearn.logisitRegression

In [ ]:
# 모델형성
logistic_model = sk$LogisticRegression()
# 모델학습
logistic_model$fit(X_train,y_train)

In [ ]:
#모델예측
logistic_pred=logistic_model$predict(X_train) # 모의고사

In [ ]:
#모델평가
logistic_r = sk.met$r2_score(y_train, logistic_pred)
logistic_r

logistic_rmse = sk.met$mean_squared_error(y_train,logistic_pred)^0.5
logistic_rmse

In [ ]:
stat = import('statsmodels')

### sklearn.lightgbm

In [ ]:
# lightgbm
sk1=import('lightgbm')

In [ ]:
# 모델형성
lgbm_model = sk1$LGBMRegressor()
#모델학습
lgbm_model$fit(X_train,y_train)

In [ ]:
#모델예측
lgbm_pred=lgbm_model$predict(X_train)

In [ ]:
#모델평가
r = sk.met$r2_score(y_train, lgbm_pred)
r
rmse = sk.met$mean_squared_error(y_train,lgbm_pred)^0.5
rmse

## 제출

In [ ]:
submission1 = lgbm_model$predict(test)
head(submission1)

In [ ]:
length(rownames(as.data.frame(submission1)))

In [ ]:
length(rownames(origin['id']))

In [ ]:
final = cbind(origin['id'], as.data.frame(submission1) )
length(rownames(final))
typeof(final)

In [ ]:
final = rename(final, '18~20_ride'='submission1')
head(final)

In [ ]:
write.csv(final,'submission_2조.csv',row.names=FALSE)

In [ ]:
model_lm <- lm(X18.20_ride ~ ., data = train)
y_pred_train_lm <- predict(model_lm, newdata = train)
rsq_train_lm <- rsquare(y_pred_train_lm, train$X18.20_ride)
rsq_adj_train_lm <- rsquare_adj(y_pred_train_lm, train$X18.20_ride, length(model_lm$coefficients) - 1)

y_pred_test_lm <- predict(model_lm, newdata = test)
rmse_test_lm <- rmse(y_pred_test_lm, train$X18.20_ride)
mae_test_lm <- mae(y_pred_test_lm, train$X18.20_ride)

In [ ]:
library(MASS)
model_lm2 <- lm(medv ~ .^2, data = df_train)
model_lm2_step <- MASS::stepAIC(model_lm2, scope = list(upper = ~ .^2, lower = ~-1))
y_pred_train_lm2 <- predict(model_lm2_step, newdata = df_train)
rsq_train_lm2 <- rsquare(y_pred_train_lm2, df_train$medv)
rsq_adj_train_lm2 <- rsquare_adj(y_pred_train_lm2, df_train$medv, length(model_lm2_step$coefficients) - 1)

y_pred_test_lm2 <- predict(model_lm2_step, newdata = df_test)
rmse_test_lm2 <- rmse(y_pred_test_lm2, df_test$medv)
mae_test_lm2 <- mae(y_pred_test_lm2, df_test$medv)

In [ ]:
library(randomForest)
model_rf <- randomForest(medv ~ ., data = df_train)
y_pred_train_rf <- predict(model_rf, newdata = df_train)
rsq_train_rf <- rsqare(y_pred_train_rf, df_train$medv)
rsq_adj_train_rf <- rsqare_adj(y_pred_train_rf, df_train$medv, 13)

y_pred_test_rf <- predict(model_rf, newdata = df_test)
rmse_test_rf <- rmse(y_pred_test_rf, df_test$medv)
mae_test_rf <- mae(y_pred_test_rf, df_test$medv)